In [1]:
import pandas as pd
import numpy as np

In [26]:
ruta1 = "DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx"
ruta2 = "DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx"

In [34]:
df = pd.read_excel(ruta1, header=None)

In [35]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13873 entries, 0 to 13872
Columns: 240 entries, 0 to 239
dtypes: float64(14), object(226)
memory usage: 25.4+ MB


,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,SURESTE,SURESTE,SURESTE,SURESTE,SURESTE,SURESTE,SURESTE,SURESTE,SURESTE,...,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3,NOROESTE 3
2,date,CO (ppm),NO (ppb),NO2 (ppb),NOX (ppb),O3 (ppb),PM10 (ug/m3),PM2.5 (ug/m3),PRS (mmHg),RAINF (mm/h),...,PM10 (ug/m3),PM2.5 (ug/m3),PRS (mmHg),RAINF (mm/h),RH (%),SO2 (ppb),SR (kW/m2),TOUT (ºC),WSR (km/h),WDR (azimutal)
3,2023-01-01 00:00:00,2.37,54.5,32.6,87.1,3,110,68,721.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-01 01:00:00,2.12,38.7,30.3,68.9,3,116,67.18,721.5,0,...,246,NaN,707.4,0,59,2.9,0,14.14,65.3,95


In [43]:
estaciones = df.iloc[1].dropna().unique()
print("Estaciones detectadas:", estaciones)

# --- Archivo de salida ---
salida = "TODAS_ESTACIONES_salida.xlsx"
writer = pd.ExcelWriter(salida, engine="openpyxl")

# --- Iterar sobre cada estación ---
for estacion in estaciones:
    # Columnas que corresponden a esta estación
    mask = df.iloc[1] == estacion
    col_indices = [0] + list(df.columns[mask])  # incluye columna de fechas
    
    # Sub-dataframe de la estación
    sub_df = df.iloc[:, col_indices]
    
    # Encabezados: fila 2 contiene nombres de variables
    sub_df.columns = sub_df.iloc[2]
    
    # Datos reales desde fila 3 en adelante
    sub_df_final = sub_df.iloc[3:].reset_index(drop=True)
    
    # Guardar como hoja
    sub_df_final.to_excel(writer, sheet_name=estacion[:31], index=False)  # nombres de hoja max 31 chars
    print(f"✅ Hoja creada: {estacion} ({sub_df_final.shape[1]} columnas)")

# --- Guardar archivo final ---
writer.close()
print("✅ Archivo final creado:", salida)

Estaciones detectadas: ['SURESTE' 'NORESTE' 'CENTRO' 'NOROESTE' 'SUROESTE' 'NOROESTE 2' 'NORTE'
 'NORESTE2' 'SURESTE2' 'SUROESTE2' 'SURESTE 3' 'SUR' 'NORTE 2' 'NORESTE 3'
 'NOROESTE 3']
✅ Hoja creada: SURESTE (16 columnas)
✅ Hoja creada: NORESTE (16 columnas)
✅ Hoja creada: CENTRO (16 columnas)
✅ Hoja creada: NOROESTE (16 columnas)
✅ Hoja creada: SUROESTE (16 columnas)
✅ Hoja creada: NOROESTE 2 (16 columnas)
✅ Hoja creada: NORTE (16 columnas)
✅ Hoja creada: NORESTE2 (16 columnas)
✅ Hoja creada: SURESTE2 (16 columnas)
✅ Hoja creada: SUROESTE2 (16 columnas)
✅ Hoja creada: SURESTE 3 (16 columnas)
✅ Hoja creada: SUR (16 columnas)
✅ Hoja creada: NORTE 2 (16 columnas)
✅ Hoja creada: NORESTE 3 (16 columnas)
✅ Hoja creada: NOROESTE 3 (16 columnas)
✅ Archivo final creado: TODAS_ESTACIONES_salida.xlsx


In [44]:
archivo_existente = "TODAS_ESTACIONES_salida.xlsx"  # el que ya generaste
archivo_nuevo = "DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx"  # el de todo 2024

# --- Mapeo entre nombres de hoja ---
mapa_estaciones = {
    "SE": "SURESTE",
    "NE": "NORESTE",
    "CE": "CENTRO",
    "NO": "NOROESTE",
    "SO": "SUROESTE",
    "NO2": "NOROESTE 2",
    "NTE": "NORTE",
    "NE2": "NORESTE2",
    "SE2": "SURESTE2",
    "SO2": "SUROESTE2",
    "SUR": "SUR",
    "NTE2": "NORTE 2",
    "SE3": "SURESTE 3",
    "NO3": "NOROESTE 3",
    "NE3": "NORESTE 3"
}

# --- Leer ambos archivos ---
dfs_existentes = pd.read_excel(archivo_existente, sheet_name=None)  # dict: nombre_hoja -> df
dfs_nuevos = pd.read_excel(archivo_nuevo, sheet_name=None)          # dict: abreviatura -> df

# --- Crear nuevo writer ---
salida = "DATOS_COMBINADOS_2023_2024_COMPLETO.xlsx"
writer = pd.ExcelWriter(salida, engine="openpyxl")

# --- Procesar estación por estación ---
for abrev, nombre_estacion in mapa_estaciones.items():
    if nombre_estacion not in dfs_existentes or abrev not in dfs_nuevos:
        print(f"⚠️ Saltando {nombre_estacion} (no encontrada en alguno de los archivos)")
        continue
    
    df_existente = dfs_existentes[nombre_estacion].copy()
    df_nuevo = dfs_nuevos[abrev].copy()

    # Asegurar que la columna de fecha tenga nombre estándar
    df_existente.rename(columns={df_existente.columns[0]: "fecha"}, inplace=True)
    df_nuevo.rename(columns={df_nuevo.columns[0]: "fecha"}, inplace=True)

    # Convertir fechas
    df_existente["fecha"] = pd.to_datetime(df_existente["fecha"], errors="coerce")
    df_nuevo["fecha"] = pd.to_datetime(df_nuevo["fecha"], errors="coerce")

    # Filtrar los datos nuevos desde 2024-08-01 en adelante
    df_nuevo_filtrado = df_nuevo[df_nuevo["fecha"] >= "2024-08-01"]

    # Concatenar
    combinado = pd.concat([df_existente, df_nuevo_filtrado], ignore_index=True)

    # Guardar en nueva hoja
    combinado.to_excel(writer, sheet_name=nombre_estacion[:31], index=False)
    print(f"✅ Combinada estación: {nombre_estacion} ({len(combinado)} registros)")

# --- Guardar archivo final ---
writer.close()
print(f"\n✅ Archivo combinado guardado como: {salida}")

✅ Combinada estación: SURESTE (17542 registros)
✅ Combinada estación: NORESTE (17542 registros)
✅ Combinada estación: CENTRO (17542 registros)
✅ Combinada estación: NOROESTE (17541 registros)
✅ Combinada estación: SUROESTE (17542 registros)
✅ Combinada estación: NOROESTE 2 (17542 registros)
✅ Combinada estación: NORTE (17542 registros)
✅ Combinada estación: NORESTE2 (17542 registros)
✅ Combinada estación: SURESTE2 (17542 registros)
✅ Combinada estación: SUROESTE2 (17542 registros)
✅ Combinada estación: SUR (17542 registros)
✅ Combinada estación: NORTE 2 (17542 registros)
✅ Combinada estación: SURESTE 3 (17542 registros)
✅ Combinada estación: NOROESTE 3 (17542 registros)
✅ Combinada estación: NORESTE 3 (17542 registros)

✅ Archivo combinado guardado como: DATOS_COMBINADOS_2023_2024_COMPLETO.xlsx
